In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import os, sys
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from tqdm import tqdm, tqdm_notebook
# sys.path.append('../../satclip/satclip/')
# from load import get_satclip
sys.path.append('../scripts/')
from loadpaths_glc import loadpaths
import data_loading_utils as dlu
import glc_prediction_utils as gpu
path_dict = loadpaths()




In [3]:
path_dict

{'home': '/Users/t.vanderplas/',
 'repo': '/Users/t.vanderplas/repos/GLC24/',
 'repo_GLC_official': '/Users/t.vanderplas/repos/GeoLifeCLEF/',
 'data_folder': '/Users/t.vanderplas/data_offline/data_geolifeclef2024/geolifeclef-2024/'}

In [6]:
dict_dfs, dict_dfs_train = dlu.load_metadata()

Columns dropped: ['taxonRank', 'geoUncertaintyInM', 'date', 'areaInM2']


In [17]:
for key, df in dict_dfs.items():
    print('\n----')
    print(f'{key}: {df.shape[0]} samples')
    print(df.columns)
    n_unique_samples = df['surveyId'].nunique()
    if 'test' not in key:
        n_unique_species = df['speciesId'].nunique()
    else:
        n_unique_species = 'NA'
    print(f'Unique samples: {n_unique_samples}')
    print(f'Unique species: {n_unique_species}')


----
df_train_pa: 1483637 samples
Index(['lon', 'lat', 'year', 'geoUncertaintyInM', 'areaInM2', 'region',
       'country', 'speciesId', 'surveyId'],
      dtype='object')
Unique samples: 88987
Unique species: 5016

----
df_train_po: 5079797 samples
Index(['publisher', 'year', 'month', 'day', 'lat', 'lon', 'geoUncertaintyInM',
       'taxonRank', 'date', 'dayOfYear', 'speciesId', 'surveyId'],
      dtype='object')
Unique samples: 3845533
Unique species: 9709

----
df_test_pa: 4716 samples
Index(['lon', 'lat', 'year', 'geoUncertaintyInM', 'areaInM2', 'region',
       'country', 'surveyId'],
      dtype='object')
Unique samples: 4716
Unique species: NA


In [31]:
df_train_pa.head()
# df_train_po['publisher'].unique()

,lon,lat,year,region,country,speciesId,surveyId
0,3.099038,43.134956,2021,MEDITERRANEAN,France,6874.0,212
1,3.099038,43.134956,2021,MEDITERRANEAN,France,476.0,212
2,3.099038,43.134956,2021,MEDITERRANEAN,France,11157.0,212
3,3.099038,43.134956,2021,MEDITERRANEAN,France,8784.0,212
4,3.099038,43.134956,2021,MEDITERRANEAN,France,4530.0,212


In [32]:
df_train_po.head()

,publisher,year,month,day,lat,lon,dayOfYear,speciesId,surveyId
0,Pl@ntNet,2019,5.0,5.0,43.74605,1.573057,125,3383.0,1
1,Pl@ntNet,2021,3.0,17.0,42.12559,0.314948,76,1152.0,2
2,Pl@ntNet,2021,6.0,5.0,48.29520,-0.934518,156,6772.0,3
3,iNaturalist.org,2021,6.0,9.0,53.63367,-2.644535,160,3318.0,4
4,iNaturalist.org,2021,4.0,1.0,49.79471,7.925086,91,3374.0,5


In [33]:
df_test_pa.head()


,lon,lat,year,region,country,surveyId
0,10.033550,57.12081,2019,CONTINENTAL,Denmark,642
1,7.333000,46.22997,2019,ALPINE,Switzerland,1792
2,1.843658,42.58006,2018,ALPINE,France,3256
3,11.720090,46.26149,2021,ALPINE,Italy,3855
4,9.361870,55.90245,2017,CONTINENTAL,Denmark,4889


In [36]:
from multiprocessing import freeze_support
import reverse_geocoder as rg

ind = 0
loc = rg.search((df_train_pa['lat'][ind], df_train_pa['lon'][ind]))
loc

Loading formatted geocoded file...


[{'lat': '43.10759',
  'lon': '3.08651',
  'name': 'Gruissan',
  'admin1': 'Languedoc-Roussillon',
  'admin2': "Departement de l'Aude",
  'cc': 'FR'}]

In [42]:
tmp = dlu.get_path_sat_patch_per_survey(1400102)

In [58]:
tmp = dlu.load_multiple_env_raster()
tmp

,surveyId,Elevation,LandCover,Soilgrid-bdod,Soilgrid-cec,Soilgrid-cfvo,Soilgrid-clay,Soilgrid-nitrogen,Soilgrid-phh2o,Soilgrid-sand,...,Bio10,Bio11,Bio12,Bio13,Bio14,Bio15,Bio16,Bio17,Bio18,Bio19
0,212,47.0,8.0,140.0,214.0,151.0,292.0,159.0,73.0,284.0,...,2958,2816,6355,933,143,395,2282,770,770,1917
1,222,6.0,12.0,120.0,225.0,101.0,94.0,379.0,58.0,650.0,...,2890,2746,7286,792,376,201,2283,1336,2076,1653
2,243,10.0,5.0,115.0,261.0,86.0,131.0,373.0,54.0,750.0,...,2888,2759,8546,1117,372,295,2986,1438,2448,1838
3,324,116.0,14.0,118.0,179.0,117.0,225.0,230.0,57.0,238.0,...,2938,2802,10719,1163,564,197,3216,1948,1948,2926
4,333,84.0,14.0,138.0,233.0,176.0,328.0,250.0,71.0,231.0,...,2927,2792,9353,1095,503,249,3186,1638,1638,2677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88982,3919553,23.0,12.0,116.0,199.0,65.0,116.0,311.0,59.0,622.0,...,2887,2752,8000,919,408,218,2505,1425,2187,1723
88983,3919592,138.0,1.0,118.0,245.0,113.0,156.0,405.0,54.0,555.0,...,2888,2748,6932,781,314,240,2242,1214,2029,1498
88984,3919620,37.0,12.0,113.0,233.0,70.0,71.0,253.0,56.0,798.0,...,2887,2746,9745,1201,440,246,3167,1718,2489,2413
88985,3919640,131.0,1.0,130.0,162.0,99.0,55.0,234.0,50.0,803.0,...,2895,2722,7301,816,390,211,2357,1336,2357,1558


In [60]:
import pytorch_lightning as pl

ModuleNotFoundError: No module named 'pytorch_lightning'

In [7]:
dict_dfs['df_test_pa']

,lon,lat,year,region,country,surveyId
0,10.033550,57.12081,2019,CONTINENTAL,Denmark,642
1,7.333000,46.22997,2019,ALPINE,Switzerland,1792
2,1.843658,42.58006,2018,ALPINE,France,3256
3,11.720090,46.26149,2021,ALPINE,Italy,3855
4,9.361870,55.90245,2017,CONTINENTAL,Denmark,4889
...,...,...,...,...,...,...
4711,5.874027,51.41978,2018,ATLANTIC,Netherlands,3915838
4712,4.147443,52.01209,2017,ATLANTIC,Netherlands,3916502
4713,16.029480,48.05149,2017,ALPINE,Austria,3917793
4714,12.400560,56.09469,2018,CONTINENTAL,Denmark,3918865


In [35]:
possible_species = dict_dfs['df_train_pa']['speciesId'].unique().astype(int)
tmp = {x: [np.random.choice(possible_species, np.random.randint(2, 20))] for x in dict_dfs['df_test_pa']['surveyId']}

In [40]:
tmp2 = gpu.convert_dict_pred_to_csv(tmp, custom_name='randomly-sampled')

Saved predictions to: /Users/t.vanderplas/data_offline/data_geolifeclef2024/predictions_glc2024/GLC24_vdplasthijs_predictions-randomly-sampled_2024-04-12_15-47.csv
